In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Data Loading

SF DBI’s development pipeline datasets use four nearly-identical data models. Parcel identificator was called “BLKLOT” or “Block Lot” in older datasets and is called APN (Assessor's Parcel Number) in the most recent datasets. The same parcel id can written with a leading zero in one dataset and without a leading zero in another one. Prior to 2014 Q3, datasets don’t have fields for affordable unit counts (total affordable units and net affordable units).

Some project records don’t include building permit id or use the placeholder “MULTIPLE” instead of actual permit references. Initially, we remove permitless projects from the dataset, than re-add them at a later stage.

In [2]:
def loadData(fileName, label, fmt = 1):
    columns = ['UNITS','NET_UNITS', 'AFF_UNITS', 'NET_AFF_UNITS', 'NAMEADDR', 'APN']
    
    converters = { 'APN': lambda x: x[4:], 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 2 or fmt == 3:
        converters = { 'BLKLOT': lambda x: x.lstrip('0'), 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 4:
        converters = { 'Block Lot': lambda x: x.lstrip('0'), 'DBI Permit': lambda x: 'N'+x.replace(',',''),
                     'Location 1': lambda x: x.split('\n')[0]}
        
    X = pd.read_csv(fileName, sep=',', parse_dates=[], infer_datetime_format=True, quotechar='"', converters=converters)
    
    if fmt == 2 or fmt == 3:
        X = X.rename(columns={"AFF_UNITS_NET": "NET_AFF_UNITS", "BLKLOT": "APN"})
    if fmt == 3:
        X = X.rename(columns={"UNITSNET": "NET_UNITS"})
    if fmt == 4:
        X = X.rename(columns={"Units": "UNITS", "Net Added Units": "NET_UNITS", "Best Stat": "BESTSTAT", 'Location 1': 'NAMEADDR', 'Block Lot': 'APN',
                             'DBI Permit': 'BP_APPLNO'})
        X['NET_AFF_UNITS'] = 0
        X['AFF_UNITS'] = 0
    if fmt == 5:
        X = X.rename(columns={"PROPUSE": "PROJECT_TYPE"})
    X = X[X['BESTSTAT'] == 'CONSTRUCTION']
    X = X[X['UNITS'] > 0]
    X = X[X['NET_UNITS'] >= 0]
    
    # Filtering records without permits
    M = X[(X['BP_APPLNO'] == 'N') | (X['BP_APPLNO'] == 'NMULTIPLE')][columns+['BP_APPLNO']]
    X = X[(X['BP_APPLNO'] != 'N') & (X['BP_APPLNO'] != 'NMULTIPLE')]
    
    # Filtering duplicated records
    D = X[X.duplicated('BP_APPLNO', keep=False)][columns+['BP_APPLNO']].sort_values('BP_APPLNO')
    X=X[~X.duplicated('BP_APPLNO')]
    
    # Set unique index by Permit ID
    X = X.set_index('BP_APPLNO')
    X.index.names = ['PERMIT_ID']
    
    X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']] = X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']].astype(int)
    
    X = X[columns]
    
    return X,M,D

# Helper Indexes

y13q4 = 0
y14q1 = 1
y14q2 = 2
y14q3 = 3
y14q4 = 4
y15q1 = 5
y15q2 = 6
y15q3 = 7
y15q4 = 8
y16q1 = 9
y16q2 = 10
y16q3 = 11
y16q4 = 12
y17q1 = 13

# Files and versions

files = [
    {'label': '2013\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv', 'format': 4},
    {'label': '2014\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv', 'format': 4},
    {'label': '2014\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv', 'format': 4},
    {'label': '2014\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv', 'format': 4},
    {'label': '2014\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_4.csv', 'format': 3},
    {'label': '2015\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_1.csv', 'format': 2},
    {'label': '2015\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_2.csv', 'format': 1},
    {'label': '2015\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_3.csv', 'format': 1},
    {'label': '2015\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_4.csv', 'format': 1},
    {'label': '2016\'Q1', 'file': 'data/SF_Development_Pipeline_2016_Q1.csv', 'format': 1},
    {'label': '2016\'Q2', 'file': 'data/SF_Development_Pipeline_2016_Q2.csv', 'format': 5},
    {'label': '2016\'Q3', 'file': 'data/SF_Development_Pipeline_2016_Q3.csv', 'format': 1},
    {'label': '2016\'Q4', 'file': 'data/SF_Development_Pipeline_2016_Q4.csv', 'format': 1},
    {'label': '2017\'Q1', 'file': 'data/SF_Development_Pipeline_2017_Q1.csv', 'format': 1}
]

# Loading Data
count = len(files)
labels = []
data = []
missing = []
duplicates = []

for f in files:
    t,m,d = loadData(f['file'], f['label'], f['format'])
    data.append(t)
    missing.append(m)
    duplicates.append(d)
    labels.append(f['label'])

# Displaying duplicated records
Printing out all records with duplicate building permit id for manual inspection.

In [3]:
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])


Duplicated records at 2013'Q4


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS           NAMEADDR      APN  \
188    2.0        1.0          0              0  1076 Hampshire St  4152016   
515    2.0        2.0          0              0  1078 Hampshire St  4152046   
299    2.0        1.0          0              0     268 Madison St  5943008   
520    1.0        1.0          0              0     268 Madison St  5943051   

         BP_APPLNO  
188  N200709193092  
515  N200709193092  
299  N200711077587  
520  N200711077587


Duplicated records at 2014'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS           NAMEADDR      APN  \
124    7.0        7.0          0              0  4132 Third Street  5260002   
657    7.0        7.0          0              0       4132 03rd St  5260003   
60     2.0        1.0          0              0  1076 Hampshire St  4152016   
627    2.0        2.0          0              0  1078 Hampshire St  4152046   
151    2.0        1.0          0              0     268 Madison St  5943008   
748    1.0        1.0          0              0     268 Madison St  5943051   

         BP_APPLNO  
124  N200707055953  
657  N200707055953  
60   N200709193092  
627  N200709193092  
151  N200711077587  
748  N200711077587


Duplicated records at 2014'Q2


...

UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS           NAMEADDR      APN  \
236    7.0        7.0          0              0  4132 Third Street  5260002   
514    7.0        7.0          0              0       4132 03rd St  5260003   
171    2.0        1.0          0              0  1076 Hampshire St  4152016   
654    2.0        2.0          0              0  1078 Hampshire St  4152046   
112    1.0        1.0          0              0     83 Panorama Dr  2821010   
668    1.0        1.0          0              0     83 Panorama Dr  2821023   
269    2.0        1.0          0              0     268 Madison St  5943008   
659    1.0        1.0          0              0     268 Madison St  5943051   
326    2.0        2.0          0              0      447 Linden St   818048   
651    2.0        2.0          0              0      443 Linden St   818049   
61   133.0      133.0          0              0   55 Laguna Street   857001   
492  191.0      191.0          0              0    218 Buchanan St  857001A   

         BP_APPLNO  
236  N200707055953  
514  N200707055953  
171  N200709193092  
654  N200709193092  
112  N200711077576  
668  N200711077576  
269  N200711077587  
659  N200711077587  
326  N200912304034  
651  N200912304034  
61   N201209059006  
492  N201209059006


Duplicated records at 2014'Q4


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS             NAMEADDR      APN  \
411      2          1          0              0    1076 HAMPSHIRE ST  4152016   
448      2          2          0              0    1078 HAMPSHIRE ST  4152046   
400      2          2          0              0        447 LINDEN ST   818048   
409      2          2          0              0        443 LINDEN ST   818049   
69      98         98          0              0         1239 TURK ST   757027   
70      98         98         98             98  1100 GOLDEN GATE AV   757025   
38     191        191        160            160      218 BUCHANAN ST  857001A   
55     133        133        160            160     55 LAGUNA STREET   857001   
53     139        139          0              0        555 FULTON ST   794028   
54     136        136          0              0        746 LAGUNA ST   794015   

         BP_APPLNO  
411  N200709193092  
448  N200709193092  
400  N200912304034  
409  N200912304034  
69   N201207104447  
70   N201207104447  
38   N201209059006  
55   N201209059006  
53   N201305036062  
54   N201305036062


Duplicated records at 2015'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS           NAMEADDR      APN  \
794      2          2          0              0  1078 HAMPSHIRE ST  4152046   
863      2          1          0              0  1076 HAMPSHIRE ST  4152016   
523      2          2          0              0      443 LINDEN ST   818049   
782      2          2          0              0      447 LINDEN ST   818048   

         BP_APPLNO  
794  N200709193092  
863  N200709193092  
523  N200912304034  
782  N200912304034

# Fixing duplicates
Re-adding duplicate records with corrected values.

In [4]:
def fix_duplicate(permitId, units, netUnits, affUnits, netAffUnits, addr, apn):
    for i in range(count):
        if len(duplicates[i][duplicates[i]['BP_APPLNO'] == permitId]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn]
        duplicates[i] = duplicates[i][duplicates[i]['BP_APPLNO'] != permitId]

fix_duplicate('N200709193092', 2, 1, 0, 0, '1076-1078 Hampshire St', '4152016')
fix_duplicate('N200711077587', 2, 1, 0, 0, '268 Madison St', '5943008')
fix_duplicate('N200707055953', 7, 7, 0, 0, '4132 03rd St', '5260003')
fix_duplicate('N200912304034', 2, 2, 0, 0, '447 Linden St', '818048')
fix_duplicate('N201209059006', 191, 191, 0, 0, '218 Buchanan St', '857001')
fix_duplicate('N200711077576', 1, 1, 0, 0, '83 Panorama Dr', '2821010')
fix_duplicate('N201207104447', 98, 98, 0, 0, '1100 GOLDEN GATE AV', '757025')
fix_duplicate('N201305036062', 139, 139, 0, 0, '555 FULTON ST', '794028')

hasDuplicates = False
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])
        hasDuplicates = True
if not hasDuplicates:
    print("No duplicates present!")

No duplicates present!


# Displaying records without Permit ID
Printing all records without Permit ID

In [5]:
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])


Records without Permit ID at 2015'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS           NAMEADDR      APN  \
648    132         14          0              0  833-881 Jamestown  4991277   

    BP_APPLNO  
648         N


Records without Permit ID at 2015'Q3


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS             NAMEADDR      APN  \
48    196        196          0              0  800 BROTHERHOOD WAY  7331003   

   BP_APPLNO  
48         N


Records without Permit ID at 2015'Q4


...

UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
1138     54         54          0              0   
1261    167        167          0              0   

                                      NAMEADDR       APN BP_APPLNO  
1138  800 BROTHERHOOD WAY (UNDER CONSTRUCTION)   7331005         N  
1261  HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION)  4591C001         N


Records without Permit ID at 2016'Q4


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
1179     49         49          0              0   
1221    110        110          0              0   

                                         NAMEADDR       APN  BP_APPLNO  
1179     800 BROTHERHOOD WAY (UNDER CONSTRUCTION)   7331003          N  
1221  HUNTERS POINT SHIPYARD (UNDER CONSTRUCTION)  4591C001  NMULTIPLE


Records without Permit ID at 2017'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
1203     36         36          0              0   
1337     86         86          0              0   
1338    107        107        106            106   

                                         NAMEADDR       APN  BP_APPLNO  
1203     800 BROTHERHOOD WAY (UNDER CONSTRUCTION)   7331003  NMULTIPLE  
1337  HUNTERS POINT SHIPYARD (UNDER CONSTRUCTION)  4591C001  NMULTIPLE  
1338                                 HUNTERS VIEW   4624031  NMULTIPLE

# Fixing records without Permit ID
Correcting records without Permit ID

In [6]:
def fix_missing(permitId, units, netUnits, affUnits, netAffUnits, addr, apn):
    for i in range(count):
        if len(missing[i][missing[i]['APN'].isin(apn)]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn[0]]
        missing[i] = missing[i][~missing[i]['APN'].isin(apn)]
        
fix_missing("N_CUSTOM_JAMESTOWN", 132, 14, 0, 0, '833-881 Jamestown', ['4991277'])
fix_missing("N_CUSTOM_BROTHERHOOD", 196, 196, 0, 0, '800 BROTHERHOOD WAY', ['7331003', '7331005'])
fix_missing("N_CUSTOM_SHIPYARD", 229, 229, 0, 0, 'HUNTERS POINT SHIPYARD, PHASE I', ['4591C001', '4624031'])

hasMissing = False
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])
        hasMissing = True
if not hasMissing:
    print("No missing present")

No missing present


# Searching for incorrect unit values
Searching projects with unit counts varying from quarter to quarter and projects with net units exceeding total units.

In [7]:
def find_incorrect(column):
    incorrect = {}
    for i in range(count-1):
        X = data[i]
        for j in range(i+1, count):
            X2 = data[j]
        
            X = X[X.index.isin(X2.index)].sort_index()
            X2 = X2[X2.index.isin(X.index)].sort_index()
            Y = X[X[column] != X2[column]]
            Y2 = X2[X[column] != X2[column]]
            if len(Y)>0:
                for index, row in Y.iterrows():
                    a = int(Y.loc[index][column])
                    b = int(Y2.loc[index][column])
                    if index not in incorrect:
                        incorrect[index] = { 'min': min(a, b), 'max': max(a,b)}
                    else:
                        incorrect[index] = { 'min': min(incorrect[index]['min'], a, b), 'max': max(incorrect[index]['max'], a, b)}
    return incorrect


incorrect = find_incorrect('UNITS')
print("Unit number fluctuation")
display(incorrect)

incorrectNet = find_incorrect('NET_UNITS')
print("Net Unit number fluctuation")
display(incorrectNet)

Unit number fluctuation


{'N200507208144': {'max': 25, 'min': 12},
 'N200605161774': {'max': 452, 'min': 447},
 'N200711077587': {'max': 80, 'min': 1},
 'N200807176988': {'max': 2, 'min': 1},
 'N200810315586': {'max': 156, 'min': 121},
 'N200810315636': {'max': 15, 'min': 13},
 'N201009140800': {'max': 320, 'min': 305},
 'N201012237367': {'max': 39, 'min': 23},
 'N201112070227': {'max': 470, 'min': 65},
 'N201204168406': {'max': 83, 'min': 50},
 'N201207124717': {'max': 806, 'min': 285},
 'N201209059006': {'max': 450, 'min': 191},
 'N201209069080': {'max': 550, 'min': 540},
 'N201209119428': {'max': 210, 'min': 4},
 'N201211073775': {'max': 271, 'min': 121},
 'N201211093966': {'max': 84, 'min': 81},
 'N201211284953': {'max': 28, 'min': 11},
 'N201212216752': {'max': 190, 'min': 167},
 'N201303273113': {'max': 75, 'min': 69},
 'N201305015894': {'max': 140, 'min': 74},
 'N201306250394': {'max': 560, 'min': 239},
 'N201306250465': {'max': 84, 'min': 77},
 'N201306280783': {'max': 132, 'min': 4},
 'N201307303137':

Net Unit number fluctuation


{'N200507208144': {'max': 25, 'min': 12},
 'N200605161774': {'max': 452, 'min': 447},
 'N200709193092': {'max': 2, 'min': 1},
 'N200711077587': {'max': 77, 'min': 1},
 'N200807176988': {'max': 2, 'min': 1},
 'N200810315586': {'max': 156, 'min': 121},
 'N200810315636': {'max': 15, 'min': 13},
 'N201009140800': {'max': 320, 'min': 305},
 'N201012237367': {'max': 39, 'min': 23},
 'N201112070227': {'max': 470, 'min': 65},
 'N201204168406': {'max': 83, 'min': 50},
 'N201207124717': {'max': 806, 'min': 285},
 'N201209059006': {'max': 450, 'min': 191},
 'N201209069080': {'max': 550, 'min': 540},
 'N201209119428': {'max': 210, 'min': 4},
 'N201211073775': {'max': 271, 'min': 121},
 'N201211093966': {'max': 84, 'min': 81},
 'N201211284953': {'max': 28, 'min': 11},
 'N201212216752': {'max': 190, 'min': 167},
 'N201303273113': {'max': 75, 'min': 69},
 'N201305015894': {'max': 140, 'min': 74},
 'N201306250394': {'max': 560, 'min': 239},
 'N201306250465': {'max': 84, 'min': 77},
 'N201306280783': {

In [8]:
def fix_units(permitId, units, column):
    for i in range(count):
        if permitId in data[i].index:
            d = data[i].loc[permitId]
            d[column] = units
            data[i].loc[permitId] = d

for k in incorrect:
    fix_units(k, incorrect[k]['max'], 'UNITS')
for k in incorrectNet:
    fix_units(k, incorrectNet[k]['max'], 'NET_UNITS')    
    
# 201 Folsom
fix_units('N201207124717', 656, 'UNITS')
fix_units('N201207124717', 656, 'NET_UNITS')

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Detection of completed buildings, p.1
Identifying completed projects as the ones that were in construction in a given quarter and are removed from the pipeline in the following quarter.

In [9]:
def buildStats(data):
    buildings = list(map(lambda x: len(x), completed))
    units = list(map(lambda x: x['UNITS'].values.sum(), completed))
    netUnits = list(map(lambda x: x['NET_UNITS'].values.sum(), completed))
    aff = list(map(lambda x: x['AFF_UNITS'].values.sum(), completed[4:]))
    netAff = list(map(lambda x: x['NET_AFF_UNITS'].values.sum(), completed[4:]))
    return { "buildings": buildings, "units": units, "netUnits": netUnits, "aff": aff, "netAff": netAff }

def printStats(stats):
    print("Buildings: {}".format(stats["buildings"]))
    print("Net Units: {}".format(stats["netUnits"]))
    print("Total Units: {}".format(sum(stats["netUnits"])))
    print("Net Affordable Units: {}".format(stats["netAff"]))
    print("Total Affordable Units: {}".format(sum(stats["netAff"])))

def contains(data, key):
    for i in range(len(data)):
        if key in data[i].index:
            return True
    return False

In [10]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    X2 = data[i+1]
    X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [18, 15, 28, 48, 19, 17, 16, 24, 71, 27, 34, 43, 50]
Net Units: [1672, 328, 732, 1872, 601, 816, 845, 656, 1674, 1272, 1151, 1933, 1062]
Total Units: 14614
Net Affordable Units: [160, 10, 34, 167, 64, 96, 175, 116, 71]
Total Affordable Units: 893


# Detection of completed buildings, p.2
Correcting for projects that re-appear in the pipeline after being removed.

In [11]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    for j in range(i+1,len(data)):
        X2 = data[j]
        X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [18, 15, 28, 43, 18, 16, 16, 24, 71, 27, 34, 41, 50]
Net Units: [1672, 328, 732, 1466, 485, 337, 845, 656, 1674, 1272, 1151, 1718, 1062]
Total Units: 13398
Net Affordable Units: [0, 10, 34, 167, 64, 96, 175, 23, 71]
Total Affordable Units: 640


# Percent of units in top 10 projects for each quartal
Here we print the top ten projects (by net new units) for each quarter and compare their contribution to the total net number of units completed in the same period. As we see below, in a typical quarter the top ten projects account for 90%+ of citywide housing production.

In [12]:
for i in range(len(labels)-1):
    topUnits = completed[i].sort_values('NET_UNITS', ascending=False).head(10)['NET_UNITS'].values.sum()
    units = completed[i]['NET_UNITS'].values.sum()
    print(labels[i+1])
    print(topUnits/units)

2014'Q1
0.995215311005
2014'Q2
0.984756097561
2014'Q3
0.959016393443
2014'Q4
0.93724420191
2015'Q1
0.983505154639
2015'Q2
0.982195845697
2015'Q3
0.992899408284
2015'Q4
0.977134146341
2016'Q1
0.902031063321
2016'Q2
0.98427672956
2016'Q3
0.966116420504
2016'Q4
0.971478463329
2017'Q1
0.931261770245


# Group Stats by Year

In [13]:
for i in range(3):
    yearCount = int(sum(stats['netUnits'][i*4:i*4+4]))
    print("Year {}:      {}".format(2014 + i, yearCount))
print("Year 2017 (Q1): {}".format(completed[len(completed) - 1]['NET_UNITS'].sum()))

Year 2014:      4198
Year 2015:      2323
Year 2016:      5815
Year 2017 (Q1): 1062


# Result Table (top 10)

In [14]:
for i in range(len(labels)-1):
    print(labels[i+1])
    display(completed[i].sort_values('NET_UNITS', ascending=False).head(10))

2014'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
PERMIT_ID                                                   
N200607207084    754        754          0              0   
N201104224606    315        315          0              0   
N201207054130    273        273          0              0   
N201012217106    115        115          0              0   
N200506246051     88         88          0              0   
N200912223711     52         52          0              0   
N201109074027     24         24          0              0   
N200701051074     19         19          0              0   
N200912183521     15         15          0              0   
N200711137944      9          9          0              0   

                          NAMEADDR      APN  
PERMIT_ID                                    
N200607207084       1401 Market St  3507041  
N201104224606       185 Channel St  8711023  
N201207054130         1155 04th St  8713001  
N201012217106  1960-1998 Market St   872005  
N200506246051       333 Fremont St  3747019  
N200912223711     63 West Point Rd  4624004  
N201109074027       1600 Market St   854001  
N200701051074         246 Ritch St  3776092  
N200912183521         1266 09th Av  1742043  
N200711137944         3135 24th St  6520036

2014'Q2


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
PERMIT_ID                                                   
N201106017202    182        182          0              0   
N201111038205     40         40          0              0   
N201110146841     38         38          0              0   
N200608290880     35         35          0              0   
N201202154236     20         20          0              0   
N200711309388      3          3          0              0   
N201005122282      2          2          0              0   
N200505031415      2          1          0              0   
N200901220624      3          1          0              0   
N201104013321      1          1          0              0   

                           NAMEADDR      APN  
PERMIT_ID                                     
N201106017202           260 05th St  3732008  
N201111038205          1501 15th St  3553054  
N201110146841  1645-1661 Pacific Av   595013  
N200608290880        1080 Sutter St   279011  
N201202154236          1717 17th St  3980007  
N200711309388   1870 Golden Gate Av  1152017  
N201005122282           35 Lloyd St  1260035  
N200505031415        530 Sanchez St  3584007  
N200901220624         253 Parker Av  1086006  
N201104013321       3210 Jackson St   973031

2014'Q3


...

UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
PERMIT_ID                                                   
N201209059006    450        450         23             23   
N201312184508    260        260         31             31   
N201207104447     98         98         98             98   
N201306250465     84         84         11             11   
N201207124725     80         80          0              0   
N201306260573     41         41          5              5   
N200912032516     35         35          5              5   
N201406138386     27         27          2              2   
N200507208144     25         25          0              0   
N201312124038     12         12          0              0   

                            NAMEADDR       APN  
PERMIT_ID                                       
N201209059006  55 LAGUNA ST (BLDG 2)   0857001  
N201312184508    1634 - 1690 PINE ST   0647007  
N201207104447    1100 GOLDEN GATE AV   0757025  
N201306250465         480 POTRERO AV  3973002C  
N201207124725       LUMINA (PLAZA C)   3746001  
N201306260573           450 HAYES ST   0808039  
N200912032516          1 FRANKLIN ST   0837003  
N201406138386          832 SUTTER ST   0281003  
N200507208144      468 CLEMENTINA ST   3732071  
N201312124038            520 09TH ST   3526005

2016'Q4


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
PERMIT_ID                                                   
N201306250394    560        560          0              0   
N201112070227    470        470          0              0   
N201409116118    202        202          0              0   
N201312265046    129        129          0              0   
N201401166475     91         91          0              0   
N201211093966     84         84          0              0   
N201311202405     72         72         11             11   
N201211284953     28         28         11             11   
N201401297354     21         21          0              0   
N201305217457     12         12          1              1   

                                NAMEADDR      APN  
PERMIT_ID                                          
N201306250394           1 HENRY ADAMS ST  3911001  
N201112070227  POTRERO1010 (BLDG 1 OF 2)  3833001  
N201409116118             1095 MARKET ST  3703059  
N201312265046               360 BERRY ST  8704004  
N201401166475    2700 ARELIOUS WALKER DR  4884027  
N201211093966               2655 BUSH ST  1052024  
N201311202405             346 POTRERO AV  3962008  
N201211284953             ONE CAPITOL AV  7148040  
N201401297354     2353 - 2347 LOMBARD ST  0512026  
N201305217457                690 PAGE ST  0843016

2017'Q1


UNITS  NET_UNITS  AFF_UNITS  NET_AFF_UNITS  \
PERMIT_ID                                                   
N201307303137    416        416         62             62   
N201404042522    200        200          0              0   
N201311222660     77         77          0              0   
N200711077587     80         77          0              0   
N200809252660     69         69          9              9   
N201306270646     40         40          0              0   
N200806164548     76         37          0              0   
N200412171712     27         27          0              0   
N201412194144     81         24          0              0   
N201401247066     22         22          0              0   

                                        NAMEADDR      APN  
PERMIT_ID                                                  
N201307303137                        350 08TH ST  3756003  
N201404042522                MISSION BAY BLOCK 7  8711031  
N201311222660             2101 & 2155 WEBSTER ST  0629037  
N200711077587                     268 MADISON ST  5943008  
N200809252660                   1450 FRANKLIN ST  0671006  
N201306270646  55 LAGUNA ST (AFF SENIOR HOUSING)  0857002  
N200806164548                        226 06TH ST  3731003  
N200412171712                     1601 LARKIN ST  0620006  
N201412194144                      690 MARKET ST  0311016  
N201401247066               233 - 237 SHIPLEY ST  3753096

# Exporting datasets

In [15]:
for i in range(count):
    fname = 'data/PipelineCleaned_'+labels[i].replace('\'','_')+'.csv'
    data[i].to_csv(fname)
for i in range(count-1):
    fname = 'data/PipelineCompleted_'+labels[i+1].replace('\'','_')+'.csv'
    completed[i].to_csv(fname)

# Total in pipeline per quartal

In [16]:
for i in range(count):
    print("{}: {}".format(labels[i], data[i]['NET_UNITS'].values.sum()))

2013'Q4: 6410
2014'Q1: 5106
2014'Q2: 6187
2014'Q3: 7246
2014'Q4: 7793
2015'Q1: 7723
2015'Q2: 8651
2015'Q3: 9186
2015'Q4: 9142
2016'Q1: 8716
2016'Q2: 8385
2016'Q3: 7598
2016'Q4: 6754
2017'Q1: 6105
